**Цель домашнего задания**

Отработать навык работы с оконными функциями на транзакционных данных.

 

**Условия**

Дано 2 csv-файла: один с транзакциями за период 01.01.2023 - 20.04.2023, второй с информацией о клиентах.

**За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’**

Необходимо в Jupyter-ноутбуке выполнить следующие пункты, используя SQLite:

**Шаг 1. Подготовительный этап**

*Аналогичные действия вы делали в прошлом домашнем задании.*

**(Балл - 2)** Необходимо скачать CSV-файл - «[transactions_for_dz2.csv](https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link)», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции

(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции

(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента

(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала

(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «[client_info.csv](https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link)», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента

(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте

(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте

(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента

(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождения клиента

**Шаг 2**

**(Балл - 2 за каждый пункт)** Написать скрипты:

- a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

- b. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
- c. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

- d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

- e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем что клиент может быть недействующим)
 

 

Сроки выполнения задания
До 16.04.2023, 23:59.

Импортируем библиотеки

In [6]:
import pandas as pd
import sqlite3

Создадим функции для подключения к базе данных, выполнения запросов и создании таблиц.

In [7]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [8]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [9]:
def sql_insert(con, entities, text_sql_insert_into_table):

    cursorObj = con.cursor()
    cursorObj.execute(text_sql_insert_into_table, entities)

## Шаг 1

Подключимся к базе данных

In [10]:
connection = create_connection('database.db')

Connection to SQLite DB successful


In [11]:
cursor = sqlite3.Cursor(connection)

Создадим таблицу transaction_bd

In [12]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS transaction_bd (
                                            "TRANSACTION_ID" INTEGER,
                                            "TX_DATETIME" NUMERIC,
                                            "CUSTOMER_ID" INTEGER,
                                            "TERMINAL_ID" INTEGER,
                                            "TX_AMOUNT" REAL NOT NULL
                                                        )
                     '''

In [13]:
execute_query(connection, sql_create_table)

Query executed successfully


Создадим таблицу customer_bd

In [14]:
sql_create_table = '''
                        CREATE TABLE IF NOT EXISTS customer_bd (
                                            "CLIENT_ID" INTEGER,
                                            "START_DT" NUMERIC,
                                            "END_DT" NUMERIC,
                                            "CLIENT_NAME" TEXT,
                                            "YEAR_BIRTH" TEXT
                                                        )
                     '''

In [15]:
execute_query(connection, sql_create_table)

Query executed successfully


Прочитаем данные из файлов и вставим их в таблицы

In [16]:
text_sql_insert_into_table = '''
                                 INSERT INTO transaction_bd(
                                                         TRANSACTION_ID,
                                                         TX_DATETIME,
                                                         CUSTOMER_ID,
                                                         TERMINAL_ID,
                                                         TX_AMOUNT
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [17]:
with open('transactions_for_dz2.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(','), text_sql_insert_into_table)
connection.commit()

In [18]:
text_sql_insert_into_table = '''
                                 INSERT INTO customer_bd(
                                                         START_DT,
                                                         END_DT,
                                                         CLIENT_NAME,
                                                         YEAR_BIRTH,
                                                         CLIENT_ID
                                                      )
                                 VALUES(?, ?, ?, ?, ?)
                             '''

In [19]:
with open('client_info.csv', encoding='utf-8') as file:
    text = file.readlines()
for line in text[1:]:
    sql_insert(connection, line.split(';'), text_sql_insert_into_table)
connection.commit()

## Шаг 2
Напишем скрипты:

In [20]:
connection.close()